In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
data = pd.read_csv('../input/london-bike-sharing-dataset/london_merged.csv',
                   parse_dates=['timestamp'], index_col='timestamp')

In [ ]:
data.head()

In [ ]:
data['hour'] = data.index.hour
data['day_of_the_week'] = data.index.dayofweek

In [ ]:
data['day_of_month'] = data.index.day
data['month'] = data.index.month

In [ ]:
data.head()

In [ ]:
sns.lineplot(x=data.index, y='cnt', data=data)

In [ ]:
data_by_month = data.resample('M').sum()
sns.lineplot(x=data_by_month.index, y='cnt', data=data_by_month)

In [ ]:
sns.pointplot(data=data, x='hour', y='cnt')

6-10 high
16-19 another spike
0-5 low 
after 19 dip due to night

In [ ]:
sns.pointplot(data=data, x='hour', y='cnt', hue='is_holiday')

diff btw is holiday

In [ ]:
sns.pointplot(data=data, x='day_of_the_week', y='cnt')

In [ ]:
train_size = int(len(data) * 0.9)
test_size = len(data) - train_size
train , test = data.iloc[:train_size], data.iloc[train_size:]

print(train.shape, test.shape)

In [ ]:
from sklearn.preprocessing import RobustScaler

f_columns = ['t1', 't2', 'hum', 'wind_speed']

f_transformer = RobustScaler()
cnt_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
cnt_transformer = cnt_transformer.fit(train[['cnt']])

train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
train['cnt'] = cnt_transformer.transform(train[['cnt']])

test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
test['cnt'] = cnt_transformer.transform(test[['cnt']])

In [ ]:
cnt_transformer.get_params()

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys= [], []
    for i in range(len(X)-time_steps):
        v = X.iloc[i:i+time_steps].values
        Xs.append(v)
        ys.append(y.iloc[i+time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
TIME_STEPS = 24

X_train, y_train = create_dataset(train, train.cnt, time_steps=TIME_STEPS)
X_test , y_test = create_dataset(test, test.cnt, time_steps=TIME_STEPS)

In [ ]:
#[samples, time_steps, n_features]
print(X_train.shape, y_train.shape)

In [ ]:
print(X_test.shape, y_test.shape)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Bidirectional(
            keras.layers.LSTM(
            units=128,
            input_shape=(X_train.shape[1], X_train.shape[2])
            )))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                   validation_split=0.1, shuffle=False)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_train_inv = cnt_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv = cnt_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv = cnt_transformer.inverse_transform(y_pred)

In [ ]:
plt.plot(y_test_inv.flatten(), marker='.', label='true')
plt.plot(y_pred_inv.flatten(),'r', label='prediciton')
plt.ylabel('Bike Count')
plt.xlabel('Time Step')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv.flatten(), marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv.flatten(), 'r', label="prediction")
plt.ylabel('Bike Count')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
y_pred_inv = (tf.squeeze(y_pred_inv))

In [ ]:
keras.metrics.mean_absolute_error(y_pred_inv, y_test).numpy()

In [ ]:
new_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=32, kernel_size=3,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=(X_train.shape[1], X_train.shape[2])),
  tf.keras.layers.Bidirectional(keras.layers.LSTM(128)),
  tf.keras.layers.Dense(1)
])

optimizer = keras.optimizers.Adam()
new_model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
new_history = new_model.fit(X_train, y_train, epochs=30, batch_size=32,
                   validation_split=0.1, shuffle=False)

## Cyclic Learning Rates

In [ ]:
def get_conv_BI_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=3,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=(X_train.shape[1], X_train.shape[2])),
  tf.keras.layers.Bidirectional(keras.layers.LSTM(128)),
  tf.keras.layers.Dense(1)
])

def compile_and_fit_model(model,optimizer, epochs, batch_size, callback):
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=optimizer,
                  metrics=["mae"])
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                   validation_split=0.1,callbacks=callback, shuffle=False)

In [ ]:
from keras.callbacks import Callback
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

class LRFinder(Callback):
    def __init__(self, min_lr, max_lr, mom=0.9, stop_multiplier=None, 
                 reload_weights=True, batches_lr_update=5):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.mom = mom
        self.reload_weights = reload_weights
        self.batches_lr_update = batches_lr_update
        if stop_multiplier is None:
            self.stop_multiplier = -20*self.mom/3 + 10 # 4 if mom=0.9
                                                       # 10 if mom=0
        else:
            self.stop_multiplier = stop_multiplier
        
    def on_train_begin(self, logs={}):
        p = self.params
        try:
            n_iterations = p['epochs']*p['samples']//p['batch_size']
        except:
            n_iterations = p['steps']*p['epochs']
            
        self.learning_rates = np.geomspace(self.min_lr, self.max_lr, \
                                           num=n_iterations//self.batches_lr_update+1)
        self.losses=[]
        self.iteration=0
        self.best_loss=0
        if self.reload_weights:
            self.model.save_weights('tmp.hdf5')
        
    
    def on_batch_end(self, batch, logs={}):
        loss = logs.get('loss')
        
        if self.iteration!=0: # Make loss smoother using momentum
            loss = self.losses[-1]*self.mom+loss*(1-self.mom)
        
        if self.iteration==0 or loss < self.best_loss: 
                self.best_loss = loss
                
        if self.iteration%self.batches_lr_update==0: # Evaluate each lr over 5 epochs
            
            if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
          
            lr = self.learning_rates[self.iteration//self.batches_lr_update]            
            K.set_value(self.model.optimizer.lr, lr)

            self.losses.append(loss)            

        if loss > self.best_loss*self.stop_multiplier: # Stop criteria
            self.model.stop_training = True
                
        self.iteration += 1
    
    def on_train_end(self, logs=None):
        if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
                
        plt.figure(figsize=(12, 6))
        plt.plot(self.learning_rates[:len(self.losses)], self.losses)
        plt.xlabel("Learning Rate")
        plt.ylabel("Loss")
        plt.xscale('log')
        plt.show()

In [ ]:
lr_finder = LRFinder(min_lr=1e-4, max_lr=1)

In [ ]:
model = get_conv_BI_model()
optimizer = keras.optimizers.Adam()
compile_and_fit_model(model, optimizer, 5, 32, lr_finder)

In [ ]:
# Implement One Cycle Policy Algorithm in the Keras Callback Class

from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy
from keras import backend as K
from keras.callbacks import *

class CyclicLR(keras.callbacks.Callback):
    
    def __init__(self,base_lr, max_lr, step_size, base_m, max_m, cyclical_momentum):
 
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.base_m = base_m
        self.max_m = max_m
        self.cyclical_momentum = cyclical_momentum
        self.step_size = step_size
        
        self.clr_iterations = 0.
        self.cm_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
        
    def clr(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)          
            return self.base_lr-(self.base_lr-self.base_lr/100)*np.maximum(0,(1-x))
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0,(1-x))
    
    def cm(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1) 
            return self.max_m
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.max_m - (self.max_m-self.base_m)*np.maximum(0,(1-x))
        
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
        if self.cyclical_momentum == True:
            if self.clr_iterations == 0:
                K.set_value(self.model.optimizer.momentum, self.cm())
            else:
                K.set_value(self.model.optimizer.momentum, self.cm())
            
            
    def on_batch_begin(self, batch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        
        if self.cyclical_momentum == True:
            self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
        
        if self.cyclical_momentum == True:
            K.set_value(self.model.optimizer.momentum, self.cm())
            

In [ ]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping

# CLR parameters

batch_size = 32
epochs = 30
max_lr = 1e-2
base_lr = 1e-3
max_m = 0.98
base_m = 0.85
cyclical_momentum = False
augment = True
cycles = 2.35

iterations = round(X_train.shape[0]/batch_size*epochs)
iterations = list(range(0,iterations+1))
step_size = len(iterations)/(cycles)

clr =  CyclicLR(base_lr=base_lr,
                max_lr=max_lr,
                max_m=max_m,
                base_m=base_m,
                step_size=step_size,
                cyclical_momentum=cyclical_momentum)
    
callbacks = [clr,
            ModelCheckpoint(filepath='best_model.h5', monitor='mae',mode='min',verbose=1,save_best_only=True)]

model = get_conv_BI_model()
optimizer = keras.optimizers.Adam()
compile_and_fit_model(model, optimizer, epochs, batch_size, callbacks)

In [ ]:
plt.plot(clr.history['iterations'], clr.history['lr'])
plt.xlabel('Training Iterations')
plt.ylabel('Learning Rate')
plt.title("One Cycle Policy")
plt.show()

In [ ]:
val_loss = history.history['val_loss']
loss = history.history['loss']
plt.plot(range(len(val_loss)),val_loss,'c',label='Validation loss')
plt.plot(range(len(loss)),loss,'m',label='Train loss')

plt.title('Training and validation losses')
plt.legend()
plt.xlabel('epochs')
plt.show()

In [ ]:
def predict(model, test):
    y_pred = model.predict(test)
    y_train_inv = cnt_transformer.inverse_transform(y_train.reshape(1, -1))
    y_test_inv = cnt_transformer.inverse_transform(y_test.reshape(1, -1))
    y_pred_inv = cnt_transformer.inverse_transform(y_pred)
    y_pred_inv = (tf.squeeze(y_pred_inv))
    return y_predict_inv

In [ ]:
y_pred = model.predict(X_test)
y_pred_inv = cnt_transformer.inverse_transform(y_pred)
y_pred_inv = (tf.squeeze(y_pred_inv))
keras.metrics.mean_absolute_error(y_pred_inv, y_test)

improved from 999 to 952

In [ ]:
plt.plot(y_test_inv.flatten(), marker='.', label='true')
plt.plot(y_pred_inv,'r', label='prediciton')
plt.ylabel('Bike Count')
plt.xlabel('Time Step')
plt.legend()
plt.show()